In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import vxcam
import vmbpy
import cv2 as cv
import time
import numpy as np

i = 0
N = 100
fps = np.zeros(N)
t0 = time.time()
with vxcam.VXCam() as vx:
    vx.pixel_format = vmbpy.PixelFormat.Mono8
    vx.auto_exposure = 'Off'
    vx.exposure_us = 20000
    vx.gain_db = 10
    
    # Padding is from [0..1] relative to the left, right, top, bottom extents
    vx.binning = 2
    vx.pad_right = 0
    vx.pad_left = 0
    vx.pad_top = 0
    vx.pad_bottom = 0

    # Start streaming, can't update again until vx.stop() is called
    vx.start()
    try:
        while True:
            
            # Pop latest image and convert to openCV format
            img = vx.pop(vmbpy.PixelFormat.Mono8) 
            if img is None: continue
            
            cv2.putText(img, f'{np.mean(fps):.0f}fps', (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, 255, thickness=2)
            cv2.imshow('', img)
            if cv2.waitKey(1) == ord('q'): break

            # FPS calculations
            fps[i % N] = 1/(time.time() - t0)
            t0 = time.time()
            i += 1
    except KeyboardInterrupt:
        pass
    finally:
        cv2.destroyAllWindows()
        cv2.waitKey(1)